Current Work in progress

In [ ]:
#Import necessary libraries
import pandas as pd
import numpy as np
import folium
import json, requests
from uszipcode import SearchEngine
import re

In [ ]:
#Import New York neighborhood data
with open('newyork_data.json') as json_data:
    nydata_json = json.load(json_data)
#Build Data Frame
ny_neighborhoods_data = nydata_json['features']
ny_neighborhoods_df = pd.DataFrame(columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'])

for data in ny_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhoods_df = ny_neighborhoods_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [ ]:
ny_postalcodes_list = []
ny_postalcodes_url = 'https://zipcode.org/city/NY/NEWYORK'
page = requests.get(ny_postalcodes_url, allow_redirects=False, timeout=30).text

test = re.findall('(?<="List_Link_Text">)\d\d\d\d\d', page)
test = re.findall('10\d\d\d', page)
test

In [ ]:
def foursquare_url(endpoint, **kwargs):
    # Foursquare Login Credentials
    foursquare_id_client = 'X'
    foursquare_id_secret = 'X'
    foursquare_id_version = '20200801'   
    
    # Construct base url with selected api endpoint and authentication credentials
    url = 'https://api.foursquare.com/v2/venues/'
    url = url + '{}?&client_id={}&client_secret={}&v={}'.format(endpoint, foursquare_id_client, 
                                                               foursquare_id_secret, foursquare_id_version) 

    if endpoint == 'search':
        search_keywords = {'radius', 'query', 'limit', 'lattitude', 'longitude', 'near', 'categoryId'}
        coordinates_flag = False
        near_flag = False
        for key, value in kwargs.items():
            # Check if valid key was provided
            if key in search_keywords:
                # Handle simple keys: radius, query and limit
                if key in {'radius', 'query', 'limit'}:
                    url = url + '&{}={}'.format(key, value)

                # Handle coordinate keys: If one of either is found 
                # and no coordinates have yet been added
                elif (key == 'lattitude' or key == 'longitude') and (not coordinates_flag):
                    # check if no near information has been provided
                    if not near_flag:
                        # Check if arguments contain both lat and long information, then add them to
                        # url and set the coordinate flag to True
                        if 'lattitude' in kwargs and 'longitude' in kwargs:
                            url = url + '&ll={},{}'.format(kwargs.get('lattitude'), kwargs.get('longitude'))
                            coordinates_flag = True
                        # Raise exception if either keyword is corrupted or missing
                        else:
                            print('Error with entering coordinates')
                            return
                    # Raise exception if keywords contain both coordinates and near information
                    else:
                        print('Coordinates and near a location used simultaneously.')
                        return
                # Handle near key: If a near key has been provided and no coordinates were entered
                elif key == 'near':
                    # Check if no coordinates were entered yet, then add near parameter to url
                    # and set near_flag to true 
                    if not coordinates_flag:   
                        url = url + '&{}={}'.format(key, value)
                        near_flag = True
                    # Raise exception if keywords contain both coordinates and near information
                    else:
                        print('Coordinates and near a location used simultaneously.')
                        return                
                elif key == 'categoryId':
                    Id_string = ",".join(value)
                    url = url + '&{}={}'.format(key, Id_string)
            else:
                print('Wrong key: ' + key)
                return
    return url